In [ ]:
!pip install jsonapi_client

In [ ]:
from jsonapi_client import Session
import pandas as pd

analysis_ids = ["MGYA00592687","MGYA00591505", "MGYA00591506"]

for analysis_id in analysis_ids:

    #########################################
    # Get all downloads for one analysis
    #########################################

    with Session("https://www.ebi.ac.uk/metagenomics/api/v1") as mgnify:
        
        dfs = []
        for r in mgnify.iterate(f'analyses/{analysis}/downloads'):
            df = pd.json_normalize(r.json)
            df['url'] = str(r.links.self)
            dfs.append(df)
        
    main_df = pd.concat(dfs)

    ##########################
    # get specific data url
    #########################

    data_type = "TSV"
    data_label = "OTUs and taxonomic assignments for SSU rRNA"

    c1 = main_df["attributes.file-format.name"] == data_type
    c2 = main_df["attributes.description.description"] == data_label

    url = main_df.loc[(c1 & c2), "url"].iloc[0]

    ##########################
    # download the data
    #########################

    data_output_folder = 'data/SSU'
    os.makedirs(data_output_folder, exist_ok=True)

    response = requests.get(url)

    if not response:
        print(f"Could not download file, got response: {response.status_code}")
        break
        
    print(url)

    data_output_path = os.path.join(data_output_folder, f"{analysis_id}.tsv")
    with open(data_output_path, "w") as f:
        f.write(response.text)
        
    # add to galaxy
    put(data_output_path)